In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:

file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_measurements.csv"

df = pd.read_csv(file_path, sep="\t")

df = df.sample(n=1000, random_state=99)


In [3]:
df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width
1921,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus impunctatus,0.603504,0.270993
4058,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus,Pterostichus lachrymosus,0.833583,0.423888


In [4]:

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")

tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False

In [5]:

def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    image = preprocess_val(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    
    return embedding.cpu().numpy().flatten()


In [6]:
df['Embedding'] = [extract_features(img) for img in tqdm(df['ImageFilePath'], desc='Extracting Embeddings')]

X = np.vstack(df['Embedding'].values)
X = np.hstack((X, df[['Length', 'Width']].values))

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(df['ScientificName'])


Extracting Embeddings: 100%|██████████| 1000/1000 [02:15<00:00,  7.40it/s]


In [7]:
df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


NaiveBayes                | Acc: 76.50% | Prec: 68.87% | Rec: 76.50% | F1: 71.59% | Bal Acc: 47.97% | MCC: 0.7500


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 81.00% | Prec: 74.72% | Rec: 81.00% | F1: 77.12% | Bal Acc: 54.77% | MCC: 0.7984
NearestNeighbor           | Acc: 83.00% | Prec: 76.03% | Rec: 83.00% | F1: 78.50% | Bal Acc: 55.40% | MCC: 0.8198


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precisio

RandomForest              | Acc: 78.00% | Prec: 69.52% | Rec: 78.00% | F1: 71.73% | Bal Acc: 46.26% | MCC: 0.7661
MLP_Baseline              | Acc: 80.00% | Prec: 73.68% | Rec: 80.00% | F1: 75.93% | Bal Acc: 54.55% | MCC: 0.7879


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": label_encoder.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width,Embedding,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
116,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus,Pterostichus melanarius,0.978987,0.531729,"[0.53170836, 0.16363397, 0.36464298, 0.0393027...",Cyclotrachelus torvus,Cyclotrachelus torvus,Pterostichus melanarius,Pterostichus melanarius,Cyclotrachelus torvus
6955,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Euryderus,Euryderus grossus,0.870635,0.600092,"[0.0028577521, 0.19530705, 0.0334059, -0.09936...",Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus


In [10]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP-traits.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP-traits-metrics.csv", index=False)